In [5]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv("6.1P_data.csv")

app = Dash()

app.layout = [
    dcc.Textarea(id='text-item', value='Enter sample size'),
    html.Button('Submit', id='text-item-button', n_clicks=0),
    dcc.Dropdown(options=['Line', 'Scatterplot', 'Distribution'], value='Line', id='graph-dropdown-item'),
    dcc.Dropdown(options=['x', 'y', 'z', 'All'], value='x', id='controls-and-dropdown-item'),
    dash_table.DataTable(id='data-table', page_size=6),
    dcc.Graph(figure={}, id='controls-and-graph')
]

@callback(
    Output('data-table', 'data'),
    Output(component_id='controls-and-graph', component_property='figure'),
    State('text-item', 'value'),
    Input('text-item-button', 'n_clicks'),
    Input(component_id='graph-dropdown-item', component_property='value'),
    Input(component_id='controls-and-dropdown-item', component_property='value')
)
def update(sample_size_text, button_state, graph_type, col_chosen):

    if button_state > 0:
        try :
            sample_size = int(sample_size_text)
            if sample_size > df.shape[0] :
                sample_size = df.shape[0]
        except ValueError :
            sample_size = df.shape[0]
    else :
        sample_size = df.shape[0]

    
    data = df.head(sample_size).to_dict('records')
    
    
    if graph_type == 'Line':
        if col_chosen == 'All':
            fig = px.line(df.head(sample_size), x='time', y='x')
            fig.add_trace(go.Scatter(x=df.head(sample_size)['time'], y=df.head(sample_size)['y'], name='y', mode='lines', showlegend=False))
            fig.add_trace(go.Scatter(x=df.head(sample_size)['time'], y=df.head(sample_size)['z'], name='z', mode='lines', showlegend=False))
        else:
            fig = px.line(df.head(sample_size), x='time', y=col_chosen)
            
    elif graph_type == 'Scatterplot':
        if col_chosen == 'All':
            fig = px.scatter(df.head(sample_size), x='time', y=['x', 'y', 'z'])
        else :
            fig = px.scatter(df.head(sample_size), x='time', y=col_chosen)
        
    elif graph_type == 'Distribution':
        if col_chosen == 'All':
            fig = px.histogram(df.head(sample_size), x='time', y=df.head(sample_size)['x'], hover_data='x')
            fig.add_trace(go.Bar(x=df.head(sample_size)['time'], y=df.head(sample_size)['y'], name='y', showlegend=False))
            fig.add_trace(go.Bar(x=df.head(sample_size)['time'], y=df.head(sample_size)['z'], name='z', showlegend=False))
            fig.update_layout(yaxis_range=[0,3])
        else :
            fig = px.bar(df.head(sample_size), x='time', y=col_chosen, hover_data=col_chosen)

        
    return data, fig

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>